In [86]:
states = {'Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia',
          'Florida','Georgia','Hawaii',
          'Idaho','Illinois','Indiana','Iowa','Kansas', 'Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan',
          'Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire', 'New Jersey', 'New Mexico', 'New York',
          'North Carolina', 'North Dakota','Ohio', 'Oklahoma','Oregon','Pennsylvania','Rhode Island','South Dakota','South Carolina',
          'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia','Washington', 'West Virginia', 'Wisconsin', 'Wyoming'}
years = [2017, 2018, 2019, 2020, 2021]

#2020'South Carolina', 2020'Tennessee',

Download PDF

In [2]:
import urllib.request
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
import requests
import shutil

for state in tqdm(states):
    for year in years:
        state_rename = state.lower().replace(' ', '-')
        if state == 'District of Columbia':
            state_rename = 'district-columbia'
        input_name = str(year)+'-'+state_rename+'-sat-suite-assessments-annual-report.pdf'
        output_name = 'US_datasets/'+str(year)+'-'+state_rename+'-sat-suite-of-assessments-annual-report.pdf'
        url = 'https://reports.collegeboard.org/pdf/'+ input_name

        r = requests.get(url, auth=('usrname', 'password'), verify=False,stream=True)
        r.raw.decode_content = True
        with open(output_name, 'wb') as f:
                shutil.copyfileobj(r.raw, f)

100%|██████████| 51/51 [06:10<00:00,  7.27s/it]


Parse PDF

In [5]:
import pdfplumber
import pandas as pd

In [6]:
def delete_comma(word):
    new_word = ''
    for letter in word:
        if letter != ',':
            new_word += letter
    return new_word

In [7]:
def number_position(line_arr):
    if len(line_arr) == 8:
        return line_arr[4], line_arr[6]
    
    elif len(line_arr) == 5:
        return line_arr[3], line_arr[4]
    
    elif len(line_arr) == 6:
        if line_arr[2][-1] == '%':
            return line_arr[4], line_arr[5]
        elif line_arr[4][-1] == '%':
            return line_arr[3], line_arr[5]
        else:
            return line_arr[3], line_arr[4]
    
    elif len(line_arr) == 7:
        if line_arr[2][-1] == '%' and line_arr[5][-1] == '%':
            return line_arr[4], line_arr[6]
        elif line_arr[2][-1] == '%' and line_arr[6][-1] == '%':
            return line_arr[4], line_arr[5]
        else:
            return line_arr[3], line_arr[5]

In [57]:
def total_scores(pdf):
    #total scores and perc
    page = pdf.pages[4]
#     page = pdf.pages[5]
    text = page.extract_text()
    text_arr = text.splitlines()
#     print(text_arr)

    total_score = []
    total_score_perc = []
    erw = []
    erw_perc = []
    math = []
    math_perc = []

    page5 = pdf.pages[5]
    text5 = page5.extract_text().splitlines()
#     print(text5)
    all_ppl = int(delete_comma(text5[8].split()[2]))
#     page3 = pdf.pages[3]
#     text3 = page3.extract_text().splitlines()
#     all_ppl = int(delete_comma(text3[7].split()[1]))
    
    for i in range(7,12):
        line_arr = text_arr[i].split()
        tot_count = line_arr[1]
        if ',' in tot_count:
            tot_count = delete_comma(tot_count)
        total_score.append(int(tot_count))
        
        erw_count, math_count = number_position(line_arr)
        if ',' in erw_count:
            erw_count = delete_comma(erw_count)
        erw.append(int(erw_count))
 
        if ',' in math_count:
            math_count = delete_comma(math_count)
        math.append(int(math_count))
    
        total_score_perc.append(round(int(tot_count)/all_ppl*100))
        erw_perc.append(round(int(erw_count)/all_ppl*100))
        math_perc.append(round(int(math_count)/all_ppl*100))
        
    
    total_score.append(max(all_ppl - sum(total_score),0))
    erw.append(max(all_ppl - sum(erw),0))
    math.append(max(all_ppl - sum(math),0))
    total_score_perc.append(max(100-sum(total_score_perc),0))
    erw_perc.append(max(100-sum(erw_perc),0))
    math_perc.append(max(100-sum(math_perc),0))
    
    return total_score, total_score_perc, erw, erw_perc, math, math_perc

In [25]:
def total_scores2017(pdf):
    #total scores and perc
#     page = pdf.pages[4]
    page = pdf.pages[5]
    text = page.extract_text()
    text_arr = text.splitlines()
#     print(text_arr)

    total_score = []
    total_score_perc = []
    erw = []
    erw_perc = []
    math = []
    math_perc = []

    page3 = pdf.pages[3]
    text3 = page3.extract_text().splitlines()
#     print(text5)
    all_ppl = int(delete_comma(text3[7].split()[1]))
    
    for i in range(7,12):
        line_arr = text_arr[i].split()
        tot_count = line_arr[1]
        if ',' in tot_count:
            tot_count = delete_comma(tot_count)
        total_score.append(int(tot_count))
        
        erw_count, math_count = number_position(line_arr)
        if ',' in erw_count:
            erw_count = delete_comma(erw_count)
        erw.append(int(erw_count))
 
        if ',' in math_count:
            math_count = delete_comma(math_count)
        math.append(int(math_count))
    
    total_score.append(max(all_ppl - sum(total_score),0))
    erw.append(max(all_ppl - sum(erw),0))
    math.append(max(all_ppl - sum(math),0))
    
    all_ppl = sum(total_score)
    for i in range(len(total_score)):
        total_score_perc.append(round(int(total_score[i])/all_ppl*100))
        erw_perc.append(round(int(erw[i])/all_ppl*100))
        math_perc.append(round(int(math[i])/all_ppl*100))
    
    return total_score, total_score_perc, erw, erw_perc, math, math_perc

In [9]:
def other_scores(pdf):
    #total scores and perc
    page = pdf.pages[5]
    text = page.extract_text().splitlines()
    fem_total = int(delete_comma(text[8].split()[3]))
    male_total = int(delete_comma(text[8].split()[4])) 

    ftotal = []
    fperc = []
    ferw = []
    ferw_perc = []
    fmath = []
    fmath_perc = []
    
    mtotal = []
    mperc = []
    merw = []
    merw_perc = []
    mmath = []
    mmath_perc = []
    
    for i in range(10,16):
        line_arr = text[i].split()
        fperc_item = int(line_arr[2][:-1])
        mperc_item = int(line_arr[3][:-1])
        fperc.append(fperc_item)
        mperc.append(mperc_item)
        ftotal.append(round(fem_total*fperc_item*0.01))
        mtotal.append(round(male_total*mperc_item*0.01))
    
    for i in range(17,23):
        line_arr = text[i].split()
        ferw_perc_item = int(line_arr[2][:-1])
        merw_perc_item = int(line_arr[3][:-1])
        ferw_perc.append(ferw_perc_item)
        merw_perc.append(merw_perc_item)
        ferw.append(round(fem_total*ferw_perc_item*0.01))
        merw.append(round(male_total*merw_perc_item*0.01))       
        
    for i in range(24,30):
        line_arr = text[i].split()
        fmath_perc_item = int(line_arr[3][:-1])
        mmath_perc_item = int(line_arr[4][:-1])
        fmath_perc.append(fmath_perc_item)
        mmath_perc.append(mmath_perc_item)
        fmath.append(round(fem_total*fmath_perc_item*0.01))
        mmath.append(round(male_total*mmath_perc_item*0.01))  
        
    return ftotal, fperc, ferw, ferw_perc, fmath, fmath_perc,mtotal, mperc, merw, merw_perc, mmath, mmath_perc

In [50]:
parsed_df = {'state': [], 'year':[],
            'tot_mean':[],'subject_mean':[],
            'total_counts':[],'total_perc':[],'total_erw':[], 'total_erw_perc':[], 'total_math':[], 'total_math_perc':[],
            'female_counts':[],'female_perc':[], 'female_erw':[], 'female_erw_perc':[], 'female_math':[], 'female_math_perc':[],
            'male_counts':[],'male_perc':[], 'male_erw':[], 'male_erw_perc':[], 'male_math':[], 'male_math_perc':[]}

In [58]:
def add_to_dict(year, state, pdf):
    parsed_df['state'] += [state]*6
    parsed_df['year'] += [year]*6
    parsed_df['tot_mean'] += [1495,1295,1095,895,695,495]
    parsed_df['subject_mean'] += [750,645,545,445,345,245]
    
    total_score, total_score_perc, erw, erw_perc, math, math_perc = total_scores(pdf)
    parsed_df['total_counts'] += total_score
    parsed_df['total_perc'] += total_score_perc
    parsed_df['total_erw'] += erw
    parsed_df['total_erw_perc'] += erw_perc
    parsed_df['total_math'] += math
    parsed_df['total_math_perc'] += math_perc
    
    ftotal, fperc, ferw, ferw_perc, fmath, fmath_perc,mtotal, mperc, merw, merw_perc, mmath, mmath_perc = other_scores(pdf)
    parsed_df['female_counts'] += ftotal
    parsed_df['female_perc'] += fperc
    parsed_df['female_erw'] += ferw
    parsed_df['female_erw_perc'] += ferw_perc
    parsed_df['female_math'] += fmath
    parsed_df['female_math_perc'] += fmath_perc
    
    parsed_df['male_counts'] += mtotal
    parsed_df['male_perc'] += mperc
    parsed_df['male_erw'] += merw
    parsed_df['male_erw_perc'] += merw_perc
    parsed_df['male_math'] += mmath
    parsed_df['male_math_perc'] += mmath_perc

#     parsed_df['female_counts'] += [0]*6
#     parsed_df['female_perc'] += [0]*6
#     parsed_df['female_erw'] += [0]*6
#     parsed_df['female_erw_perc'] += [0]*6
#     parsed_df['female_math'] += [0]*6
#     parsed_df['female_math_perc'] += [0]*6
    
#     parsed_df['male_counts'] += [0]*6
#     parsed_df['male_perc'] += [0]*6
#     parsed_df['male_erw'] += [0]*6
#     parsed_df['male_erw_perc'] += [0]*6
#     parsed_df['male_math'] += [0]*6
#     parsed_df['male_math_perc'] += [0]*6

In [29]:
# parsed_df

In [75]:
path = 'US_datasets/'+str(2021)+'-'+'alabama'+'-sat-suite-of-assessments-annual-report.pdf'
pdf = pdfplumber.open(path)
page = pdf.pages[3]
text = page.extract_text()
text_arr = text.splitlines()

In [78]:
text_arr[22].split()

['Male', '740', '43%', '1184', '595', '589', '69%', '86%', '71%', '12%']

In [79]:
def get_mean(pdf, mean_df, year, state):
    page = pdf.pages[3]
    text = page.extract_text()
    text_arr = text.splitlines() 
    tot_line = text_arr[7].split()
    fem_line = text_arr[21].split()
    male_line = text_arr[22].split()
    
    mean_df['states'].append(state)
    mean_df['year'].append(year)
    mean_df['tot_mean'].append(tot_line[2])
    mean_df['erw_mean'].append(tot_line[3])
    mean_df['math_mean'].append(tot_line[4])
    mean_df['tot_fem'].append(fem_line[3])
    mean_df['erw_fem'].append(fem_line[4])
    mean_df['math_fem'].append(fem_line[5])
    mean_df['tot_male'].append(male_line[3])
    mean_df['erw_male'].append(male_line[4])
    mean_df['math_male'].append(male_line[5])

In [87]:
mean_df = {'states':[], 'year':[], 'tot_mean':[], 'erw_mean':[], 'math_mean':[],
          'tot_fem':[], 'erw_fem':[], 'math_fem':[],
          'tot_male':[], 'erw_male':[], 'math_male':[]}

In [88]:
for state in tqdm(states):
    print(state)
    for year in years:
        state_rename = state.lower().replace(' ', '-')
        path = 'US_datasets/'+str(year)+'-'+state_rename+'-sat-suite-of-assessments-annual-report.pdf'
        pdf = pdfplumber.open(path)
        get_mean(pdf, mean_df, year, state)
#         add_to_dict(year, state, pdf)



  0%|          | 0/51 [00:00<?, ?it/s]

Ohio


  2%|▏         | 1/51 [00:03<02:54,  3.49s/it]

Delaware


  4%|▍         | 2/51 [00:07<02:56,  3.60s/it]

...

 96%|█████████▌| 49/51 [02:48<00:06,  3.47s/it]

North Dakota


 98%|█████████▊| 50/51 [02:51<00:03,  3.29s/it]

Wisconsin


100%|██████████| 51/51 [02:54<00:00,  3.42s/it]


In [61]:
df = pd.DataFrame.from_dict(parsed_df)
# df.to_csv('US_data.csv')

In [65]:
res = pd.concat([us,df],ignore_index=True)
res.to_csv('US_data.csv')

Get Mean

In [89]:
mdf = pd.DataFrame.from_dict(mean_df)
mdf.to_csv('sat_mean.csv')